### Comandos básicos do SQL

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Criando a conexão
import sqlite3
con = sqlite3.connect('BaseDados.db')

In [3]:
# Criando um cursor
cur = con.cursor()

In [4]:
# Criando uma função para consultar os dados
def executa_sql(comando):
    cur.execute(comando)
    resultado = cur.fetchall()
    resultado = pd.DataFrame(resultado)
    if resultado.shape[1] > 0:
        resultado.columns = [i[0] for i in cur.description]
    print(resultado.shape)
    display(resultado.head())
    return resultado

In [5]:
# Visualizando a base
resultado_sql = executa_sql('SELECT * FROM dados')

(43, 11)


,index,id_aluno,nome_aluno,cod_matricula,e-mail,acesso_plataforma,acesso_liberado,dias_ultimo_acesso,nr_prova,prova_feita,nota_prova
0,0,1,Maria Eduarda da Rocha,38273,maria@hashtag.com,0,1,12,1,1,9.0
1,1,2,Bárbara da Cunha,63546,barbara@hashtag.com,0,1,2,1,1,10.0
2,2,3,Kevin Melo,80515,kevin@hashtag.com,1,1,3,1,1,7.0
3,3,4,Pedro Henrique da Costa,68004,pedro@hashtag.com,1,1,1,1,1,4.0
4,4,5,Mirella Viana,28421,mirella@hashtag.com,1,1,1,1,1,7.0


### Selecionando os 5 primeiros e 5 últimos registros

In [6]:
# Fazendo o GROUP BY do pandas para visualizar os alunos com maior média
resultado_sql.groupby('nome_aluno')['nota_prova'].mean().reset_index()

,nome_aluno,nota_prova
0,Antônio Azevedo,6.000000
1,Bárbara Freitas,8.000000
2,Bárbara da Cunha,10.000000
3,Cauã Porto,NaN
4,Eloah Aragão,7.500000
5,Francisco Pires,NaN
6,Gabriela Costela,8.000000
7,Isabelly Souza,7.000000
8,Júlia Pinto,8.000000
9,Kevin Melo,7.000000


In [7]:
# Salvando essa informação em uma variável
media = resultado_sql.groupby('nome_aluno')['nota_prova'].mean().reset_index()
# media = media[media.nota_prova.notnull()]
media = media.sort_values('nota_prova',ascending=True)

In [8]:
# Verificando os 5 primeiros e 5 últimos registros com o pandas
media.head(5)

,nome_aluno,nota_prova
19,Pedro Henrique da Costa,4.0
0,Antônio Azevedo,6.0
11,Laura Melo,6.0
16,Maria Ferreira,6.5
18,Mirella Viana,7.0


**Agora vamos usar o SQL para visualizar essa mesma informação**

### O GROUP BY NO SQL
- O `GROUP BY` permite que a gente agregue os valores das colunas
- Para usar o group by no SQL, vamos passar a `função de agregação` na seleção das colunas e depois usamos o `GROUP BY` após o `WHERE` (quando existir) informando as colunas pelas quais iremos agrupar

In [9]:
# Relembrando o GROUP BY
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula')

(15, 4)


,nome_aluno,cod_matricula,media_prova,qtd_provas
0,Antônio Azevedo,29022,6.0,1
1,Bárbara Freitas,19442,8.0,2
2,Bárbara da Cunha,63546,10.0,1
3,Eloah Aragão,65749,7.5,4
4,Gabriela Costela,21262,8.0,1


In [10]:
display(resultado_sql)

,nome_aluno,cod_matricula,media_prova,qtd_provas
0,Antônio Azevedo,29022,6.000000,1
1,Bárbara Freitas,19442,8.000000,2
2,Bárbara da Cunha,63546,10.000000,1
3,Eloah Aragão,65749,7.500000,4
4,Gabriela Costela,21262,8.000000,1
5,Isabelly Souza,31859,7.000000,1
6,Júlia Pinto,47086,8.000000,2
7,Kevin Melo,80515,7.000000,1
8,Laura Melo,50966,6.000000,1
9,Lívia Jesus,22284,8.000000,1


### O ORDER BY NO SQL
- O `ORDER BY` vai permitir a ordenação do resultado
    - ASC: ascendente
    - DESC: descendente
- Vamos passar o `ORDER BY` logo após o `GROUP BY` (quando existir)

In [11]:
# Relembrando o ORDER BY
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc')

(15, 4)


,nome_aluno,cod_matricula,media_prova,qtd_provas
0,Bárbara da Cunha,63546,10.0,1
1,Bárbara Freitas,19442,8.0,2
2,Gabriela Costela,21262,8.0,1
3,Júlia Pinto,47086,8.0,2
4,Lívia Jesus,22284,8.0,1


### O TOP / LIMIT
- Limitar o número de registros que vamos retornar na consulta

In [12]:
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc \
                            LIMIT 5')

(5, 4)


,nome_aluno,cod_matricula,media_prova,qtd_provas
0,Bárbara da Cunha,63546,10.0,1
1,Bárbara Freitas,19442,8.0,2
2,Gabriela Costela,21262,8.0,1
3,Júlia Pinto,47086,8.0,2
4,Lívia Jesus,22284,8.0,1


### O HAVING NO SQL
- Não conseguimos usar o `WHERE` com funções de agregação pois ele é executado antes da agregação
    - A ordem de processamento de uma query é:
            - FROM > WHERE > GROUP BY > HAVING > SELECT > ORDER BY
    - Para resolver isso, vamos utilizar o `HAVING`

In [13]:
# Utilizando o WHERE para selecionar apenas alunos com média maior que 5
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas FROM dados \
                            WHERE nota_prova IS NOT NULL AND AVG(nota_prova) > 5 \
                            GROUP BY nome_aluno,cod_matricula \
                            ORDER BY AVG(nota_prova) desc \
                            LIMIT 5')

OperationalError: misuse of aggregate: AVG()

In [ ]:
# Trocando o WHERE pelo HAVING
resultado_sql = executa_sql('SELECT nome_aluno, cod_matricula, AVG(nota_prova) as media_prova, \
                            count(nota_prova) as qtd_provas FROM dados \
                            WHERE nota_prova IS NOT NULL \
                            GROUP BY nome_aluno,cod_matricula \
                            HAVING AVG(nota_prova) > 7 \
                            ORDER BY AVG(nota_prova) desc')

(8, 4)


,nome_aluno,cod_matricula,media_prova,qtd_provas
0,Bárbara da Cunha,63546,10.0,1
1,Bárbara Freitas,19442,8.0,2
2,Gabriela Costela,21262,8.0,1
3,Júlia Pinto,47086,8.0,2
4,Lívia Jesus,22284,8.0,1


In [ ]:
display(resultado_sql)

,nome_aluno,cod_matricula,media_prova,qtd_provas
0,Bárbara da Cunha,63546,10.000000,1
1,Bárbara Freitas,19442,8.000000,2
2,Gabriela Costela,21262,8.000000,1
3,Júlia Pinto,47086,8.000000,2
4,Lívia Jesus,22284,8.000000,1
5,Melissa Ribeiro,38438,8.000000,2
6,Maria Eduarda da Rocha,38273,7.666667,3
7,Eloah Aragão,65749,7.500000,4
